<a href="https://colab.research.google.com/github/vivianconrad/neural-networks-and-deep-learning/blob/main/Human_Facial_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import packages
import numpy as np
import cv2
from keras.preprocessing import image
import time
from google.colab.patches import cv2_imshow
from keras.models import model_from_json

In [ ]:
# Load human face cascade file using cv2.CascadeClassifier built-in function
# cv2.CascadeClassifier([filename]) 
face_cascade_classifier = cv2.CascadeClassifier('haarcascade_frontalface.xml')

In [ ]:
# model_from_json : Parses a JSON model configuration string and returns a model instance
face_model = model_from_json(open("facial_expression.json", "r").read())

In [ ]:
# load the face expression trained model
face_model.load_weights('facial_expression.h5') 

In [ ]:
# seven standard expressions
expressions = ('Angry:', 'Disgust:', 'Fear:', 'Happy:', 'Sad:', 'Surprise:', 'Neutral:')

In [ ]:
# load the video for facial expression recognition
video = cv2.VideoCapture('video_4_vc.mp4')

In [ ]:
frame = 0

while(True):

	ret, img = video.read()
  
  # Resize the frame using cv2.resize built-in function
	# cv2.resize(capturing, output image size, x scale, y scale, interpolation)
	img = cv2.resize(img, (640, 360))
 
	img = img[0:1200,:]

  # Convert RGB to gray using cv2.COLOR_BGR2GRAY built-in function
	# BGR (bytes are reversed)
	# cv2.cvtColor: Converts image from one color space to another
	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Detect objects(faces) of different sizes using cv2.CascadeClassifier.detectMultiScale
  # cv2.CascadeClassifier.detectMultiScale(gray, scaleFactor, minNeighbors)
   
  # scaleFactor: Specifies the image size to be reduced
  # Faces closer to the camera appear bigger than those faces in the back.
    
  # minNeighbors: Specifies the number of neighbors each rectangle should have to retain it
  # Higher value results in less detections but with higher quality
	faces = face_cascade_classifier.detectMultiScale(gray, 1.3, 5)

	for (x,y,w,h) in faces:

		if w > 130: #ignore small faces

			#cv2.rectangle(img,(x,y),(x+w,y+h),(64,64,64),2)
			
			face_detected = img[int(y):int(y+h), int(x):int(x+w)] 

			face_detected = cv2.cvtColor(face_detected, cv2.COLOR_BGR2GRAY)

			face_detected = cv2.resize(face_detected, (48, 48))
			
			img_pixels = image.img_to_array(face_detected)
   
			img_pixels = np.expand_dims(img_pixels, axis = 0)
   
			#pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]
			img_pixels /= 255 

      #the probabilities of 7 expressions is stored in predictions
			predictions = face_model.predict(img_pixels) 

			max_index = np.argmax(predictions[0])
			
			# copy the image in overlay
			overlay = img.copy()
   
      # higher value of opacity, the background will be of lower color
			opacity = 0.2

			cv2.rectangle(img,(x+w+10,y-25),(x+w+150,y+115),(64,64,64),cv2.FILLED)
   
			cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)
			
			# face and expressions are connected
			cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255,255,255),1)
   
			cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255,255,255),1)
			
			expression = ""
			for i in range(len(predictions[0])):
     
				expression = "%s %s%s" % (expressions[i], round(predictions[0][i]*100, 2), '%')
				
				"""if i != max_index:
					color = (255,0,0)"""
					
				color = (0,255,255)
        
				cv2.putText(img, expression, (int(x+w+15), int(y-12+i*20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
	
	cv2_imshow(img)
	
	frame = frame + 1
	
	if frame > 227:
		break
	
	if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
		break

# Close the capturing device
video.release()

# Close all windows
cv2.destroyAllWindows()